# Databricks Delta Share to Snowflake - Initial Load Notebook

In [ ]:
!pip install delta-sharing  # Install the delta-sharing library

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
import delta_sharing
import os

In [ ]:
# Download the share profile
stage_path = "@HOL_ICE_DB.PUBLIC.DELTA_SHARE_STAGE/config.share"
local_dir  = "/tmp"
session.file.get(stage_path, local_dir)
profile_file = os.path.join(local_dir, "config.share")

In [ ]:
# Instantiate the delta‐sharing client
client = delta_sharing.SharingClient(profile_file)

In [ ]:
# Load the shared table into a pandas DataFrame
pdf = delta_sharing.load_as_pandas(f"{profile_file}#demo_delta_share.default.customers")

In [ ]:
# Convert to Snowpark DF and write out
sp_df = session.create_dataframe(pdf)
sp_df.write.save_as_table("DELTA_SHARE_LOADED", mode="append")

In [ ]:
# Define your target table (with database & schema if needed)
target_table = "hol_ice_db.PUBLIC.DELTA_SHARE_LOADED"

In [ ]:
# Overwrite (or append) that table with your DataFrame contents
sp_df.write.save_as_table(target_table, mode="append")

In [ ]:
# (Optional) Verify in Snowflake via SQL
print(session.sql(f"SELECT COUNT(*) AS CNT FROM {target_table}").collect())